In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#هذا الكود عشان يشتغل الجي بي يو حق كاقل

!ln -s /usr/local/cuda-11.0/targets/x86_64-linux/lib/libcusolver.so.10 /usr/local/cuda-11.0/targets/x86_64-linux/lib/libcusolver.so.11

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import nltk
import re
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize
import string
import numpy as np
from nltk.corpus import stopwords
import tensorflow as tf
import tensorflow.keras as keras
import transformers
from transformers import BertTokenizer, TFBertForSequenceClassification
from wordcloud import WordCloud, STOPWORDS

In [ ]:
df_train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df_test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
# function taken and modified 
# from https://towardsdatascience.com/cleaning-text-data-with-python-b69b47b97b76

stopwords = set(STOPWORDS)
stopwords.update(["nan"])

def heavy_text_clean(x):
    # first we lowercase everything
    x = x.lower()
    x = ' '.join([word for word in x.split(' ') if word not in stopwords])
    # remove unicode characters
    x = x.encode('ascii', 'ignore').decode()
    x = re.sub(r'https*\S+', ' ', x)
    x = re.sub(r'http*\S+', ' ', x)
    # then use regex to remove @ symbols and hashtags
    x = re.sub(r'@\S', '', x)
    x = re.sub(r'#\S+', ' ', x)
    x = re.sub(r'\'\w+', '', x)
    x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
    x = re.sub(r'\w*\d+\w*', '', x)
    x = re.sub(r'\s{2,}', ' ', x)
    x = re.sub(r'\s[^\w\s]\s', '', x)
    # remove single letters and numbers surrounded by space
    x = re.sub(r'\s[a-z]\s|\s[0-9]\s', ' ', x)
    return x

In [ ]:
df_train['cleaned_text'] = df_train.text.apply(heavy_text_clean)
df_test['cleaned_text'] = df_test.text.apply(heavy_text_clean)

In [ ]:
def balancing(dataframe):
    class_0,class_1 = dataframe.target.value_counts()
    df_0 = dataframe[dataframe['target']== 0]
    df_1 = dataframe[dataframe['target']== 1]
    df_1_over = df_1.sample(class_0,replace=True)
    df_train_over = pd.concat([df_0,df_1_over],axis =0)
    return df_train_over

In [ ]:
df_balanced = balancing(df_train)

In [ ]:
df_balanced.tail()

In [ ]:
df_shuffled = df_balanced.sample(frac = 1)  
#هذا الكود عشان بعد مادمجت قواعد البانات صار كل التارقت صفر فوق و الواحد تحت .. فهذا الكود يخلطهم عشان المودل تكون كويسه

In [ ]:
df_shuffled.tail()

In [ ]:
target = df_shuffled.pop('target')#series

In [ ]:
text_train = df_shuffled.pop('cleaned_text') #series

In [ ]:
text_test = df_test.pop('cleaned_text')

### Lemmatization


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
train_list = []
for word in text_train:
    tokens = word_tokenize(word)
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    train_list.append(' '.join(lemmatized))

In [ ]:
# test_list = []
# for word in text_test:
#     tokens = word_tokenize(word)
#     lemmatizer = WordNetLemmatizer()
#     lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
#     test_list.append(' '.join(lemmatized))

### BERT

In [ ]:
tar = np.array(target)

In [ ]:
hi = zip(train_list,tar)

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

In [ ]:
def convert_example_to_feature(review):
    return bert_tokenizer.encode_plus(review, 
                add_special_tokens = True,     # add [CLS], [SEP]
                max_length = 250,              # max length of the text that can go to BERT
                padding='max_length',
                truncation=True,
                return_attention_mask = True,  # add attention mask to not focus on pad tokens
              )

In [ ]:
# map to the expected input to TFBertForSequenceClassification
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_examples(ds):
  # prepare list, so that we can build up final TensorFlow dataset from slices.
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    for review, label in ds:
        bert_input = convert_example_to_feature(review)
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)

In [ ]:
ds_train_encoded = encode_examples(hi).shuffle(1000).batch(8)

In [ ]:
ds_train_encoded

In [ ]:
# map to the expected input to TFBertForSequenceClassification
def map_example_to_dict_test(input_ids, attention_masks, token_type_ids):
    return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }

def encode_examples_test(ds):
  # prepare list, so that we can build up final TensorFlow dataset from slices.
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    for review in ds:
        bert_input = convert_example_to_feature(review)
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list)).map(map_example_to_dict_test)

In [ ]:
ds_test_encoded = encode_examples_test(text_test).batch(8)

In [ ]:
ds_test_encoded

## Training Bert

In [ ]:
# log_dir = './sentiment-analysis-using-bert-keras/tensorboard_data/tb_bert'
model_save_path = './sentiment-analysis-using-bert-keras/models/bert_model.h5'

path = "./sentiment-analysis-using-bert-keras/models/"

## Initialize pre-built BERT-based classifier from transformers
bert_model = TFBertForSequenceClassification.from_pretrained(
    'bert-base-uncased', num_labels=2)

In [ ]:
#recommended learning rate for Adam 5e-5, 3e-5, 2e-5
learning_rate = 2e-5
# multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 4

# choosing Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

bert_model.compile(loss=loss,
                   optimizer=optimizer,
                   metrics=metric)

In [ ]:
history = bert_model.fit(ds_train_encoded,epochs=10)

In [ ]:
pred = bert_model.predict(ds_test_encoded,verbose=1)

In [ ]:
pred_labels = np.argmax(pred.logits, axis=1)

In [ ]:
pred_labels[:90]

In [ ]:
# array([1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
#        0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
#        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
#        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
#        1, 1])
# array([1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
#        0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
#        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
#        0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
#        1, 1])

In [ ]:
df_test.insert(4,"target", pred_labels)

In [ ]:
df_test.head()

In [ ]:
output = df_test[['id','target']]

In [ ]:
output.to_csv('submission.csv', index=False)